### Como obter as informações de Formulários Instantâneos

Inicialmente, para os testes, fizemos um formulário teste no colégio Aldeia; Tudo que eu vou obter é disso:

Vamos usar a Graph API para ver as requisições e fazer um código para obter os mesmos resultados

Para a Graph API você tem que ter essas permissões aqui:

<p align="center">
<img src="Imagens/permissoes_siq.png" width="400">
</p>

- Para você conseguir alguma possível requisição que você não tenha, você tem que escrever, selecionar as suas permissões e regenerar o token.

- Em tese esse link aqui era para falar sobre: [link](https://developers.facebook.com/docs/marketing-api/guides/lead-ads/create?locale=pt_BR)
    - A realidade é que ele ensina via _page_id_; (bizuleo)
        - No face ads, os formulários criados são associados à uma página e em tese pelo link ele te ensina a criar via API
        - Eu não consegui acessar os formulários via Graph API desse jeito - Ficava falando que eu não tinha "autoridade"
    - Como o nosso objetivo é consultar o formulário no conjunto de anúncios para ver as respostas, nosso caminho é outro: [leadgen_forms](https://developers.facebook.com/docs/graph-api/reference/page/leadgen_forms/) - Aqui tem parâmetros do `leadgen_forms`:

Mesmo aí na documentação não tem tudo lá do Graph API, então para isso eu pedi para o Chat GPT apresentar os parâmetros, talvez te ajude (me ajudou pelo menos, e o que ele falou eu certifiquei):

<p align="center">
  <img src="Imagens/tabela_chat.png" width="800">
</p>

Se você quiser ter uma noção do que eu obtive na Graph API, foi assim (presta atenção nas permissões também):

<p align="center">
<img src="Imagens/permissoes_siq_2.png" width="1200">
</p>

- Beleza, mas o que importa pra gente mesmo são as respostas dos leads, que estão em `leads`;
    - Dentro de `leads`, tem esses parâmetros aqui:
        <p align="center">
        <img src="Imagens/leads_fields.png" width="200">
        </p>
    - As respostas ficam em `field_data` e você pode acessar também quando respondeu em `created_time`;
    - Como não temos respostas (porque eu tive que desativar tudo pra não gastar dinheiro) no meu ele printou vazio, mas era para sair algo do tipo:

    <p align="center">
    <img src="Imagens/field_data.png" width="600">
    </p>

Ok. Agora que já conhecemos os parâmetros, vamos implementar a coleta em Python:


Bibliotecas

In [2]:
!pip install facebook-business

In [3]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.lead import Lead
from facebook_business.adobjects.leadgenform import LeadgenForm
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adcreative import AdCreative
import pandas as pd

Acessos(vou deixar assim mesmo)

Primeiro em tese, você teria que achar o `form_id`; Para isso eu fiz um código pra tu achar:

In [4]:
ACCESS_TOKEN = 'EAAHDFQfaZAG8BO6Deelkbp2nH3OM6FqI8tUNmcGICMNoXo45qLftUgeQKE6BOm11tjqFfaybQV1wsC4q9xBIkZBmflCVsRENyixXneRzk3LEYqhYYeVglXqOnJ2uw0dww8WswSR0mMpkVZA0I7cCiWal5fOFdiKoeIclZCZCgGn5gGQSTgkIt93KHWO1ezO8hoGN3NATXyEJmXoc0sQZDZD'
AD_ACCOUNT_ID = "act_410857323915468"
FacebookAdsApi.init(access_token=ACCESS_TOKEN)
account = AdAccount(AD_ACCOUNT_ID)

campaigns = account.get_campaigns(fields=['id', 'name', 'status'])

for campaign in campaigns:
    adsets = campaign.get_ad_sets(fields=['id', 'name', 'status']) or []

    for adset in adsets:
        ads = adset.get_ads(fields=['id', 'name', 'creative']) or []

        for ad in ads:
            creative_id = ad.get('creative', {}).get('id')
            if not creative_id:
                continue

            creative = AdCreative(creative_id).api_get(fields=['object_story_spec'])
            form_id = (
                creative.get('object_story_spec', {})
                .get('lead_gen_form_id')
            )

            if form_id:
                print(f"\nCampanha: {campaign['name']} (ID: {campaign['id']})")
                print(f"AdSet: {adset['name']} (ID: {adset['id']})")
                print(f"Anúncio: {ad['name']} (ID: {ad['id']})")
                print(f"Form ID: {form_id}")


FacebookRequestError: 

  Message: Call was not successful
  Method:  GET
  Path:    https://graph.facebook.com/v22.0/act_410857323915468/campaigns
  Params:  {'fields': 'id,name,status', 'summary': 'true'}

  Status:  400
  Response:
    {
      "error": {
        "message": "Error validating access token: The session is invalid because the user logged out.",
        "type": "OAuthException",
        "code": 190,
        "error_subcode": 467,
        "fbtrace_id": "A7VUYxuneFUweSr9Z73SLob"
      }
    }


Deu erro porque eu excedi o número de requisições; mas basicamente o caminho é:

AdAccount → Campaigns → AdSets → Ads → Creative → object_story_spec → lead_gen_form_id

In [14]:
ACCESS_TOKEN = 'EAAHDFQfaZAG8BO6Deelkbp2nH3OM6FqI8tUNmcGICMNoXo45qLftUgeQKE6BOm11tjqFfaybQV1wsC4q9xBIkZBmflCVsRENyixXneRzk3LEYqhYYeVglXqOnJ2uw0dww8WswSR0mMpkVZA0I7cCiWal5fOFdiKoeIclZCZCgGn5gGQSTgkIt93KHWO1ezO8hoGN3NATXyEJmXoc0sQZDZD'
FacebookAdsApi.init(access_token=ACCESS_TOKEN)
FORM_ID = '1019924176190315'
# Objeto do formulário
form = LeadgenForm(FORM_ID)

In [15]:
# Obter status e perguntas do formulário
form_data = form.api_get(fields=[
    LeadgenForm.Field.name,
    LeadgenForm.Field.status,
    LeadgenForm.Field.questions
])

print(f"Nome do formulário: {form_data['name']}")
print(f"Status: {form_data['status']}")
print("Perguntas:")
for question in form_data['questions']:
    print(f" - {question['label']} (type: {question['type']})")

Nome do formulário: Formulário Teste
Status: ACTIVE
Perguntas:
 - Email (type: EMAIL)
 - Full name (type: FULL_NAME)
 - Phone number (type: PHONE)
 - City (type: CITY)


Respostas dos Leads(não vai printar nada)

In [21]:
leads = form.get_leads(fields=[
    Lead.Field.created_time,
    Lead.Field.field_data
])

data = []
for lead in leads:
    row = {'created_time': lead['created_time']}
    for field in lead['field_data']:
        field_name = field.get('name')
        field_value = field.get('values', [''])[0]
        row[field_name] = field_value
    data.append(row)

df = pd.DataFrame(data)
df['created_time'] = pd.to_datetime(df['created_time']).dt.date
df

,created_time,email,full_name,phone_number,city
0,2025-03-31,rafaelimaa3@gmail.com,Rafael Lima,,Belo Horizonte


In [23]:
!pip install flask

Mandar as info via Webhook